<a href="https://colab.research.google.com/github/DavidVollendroff/DS-Unit-2-Linear-Models/blob/master/module4-logistic-regression/LS_DS_214_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [ ] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [ ] Begin with baselines for classification.
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [ ] Get your model's test accuracy. (One time, at the end.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [0]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [0]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [0]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [0]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

In [0]:
# Make burrito type categorical
df['Burrito'] = pd.Categorical(df['Burrito'])

# One-hot encode burrito type
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
burrito_types = encoder.fit_transform(df['Burrito'])
df = pd.concat((df, burrito_types), axis=1).drop('Burrito', axis=1)


In [0]:
# Convert dates to datetime object
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)

In [0]:
# Convert 'Great' boolean to 0 or 1
df.Great = df.Great.astype(int)

# change 'x' and NaN to 0 and 1 in ingredient columns
df = df.replace('x', 1)

In [0]:
# tidy the 'Chips' column
df = df.replace('X', 1)
df = df.replace('No', 0)
df = df.replace('Yes', 1)

In [0]:
ingredients = ['Beef', 'Pico', 'Guac', 'Cheese', 'Fries', 'Sour cream', 'Pork', 'Chicken', 'Shrimp', 'Fish',
              'Rice', 'Beans', 'Lettuce', 'Tomato', 'Bell peper', 'Carrots', 'Cabbage', 'Sauce', 'Salsa.1',
              'Cilantro', 'Onion', 'Taquito', 'Pineapple', 'Ham', 'Chile relleno', 'Nopales', 'Lobster',
              'Queso', 'Egg', 'Mushroom', 'Bacon', 'Sushi', 'Avocado', 'Corn', 'Zucchini', 'Chips']
df[ingredients] = df[ingredients].fillna(0)

In [0]:
df.isnull().sum()

Date                     0
Yelp                   334
Google                 334
Chips                    0
Cost                     7
                      ... 
Burrito_Asada            0
Burrito_California       0
Burrito_Carnitas         0
Burrito_Other            0
Burrito_Surf & Turf      0
Length: 63, dtype: int64

In [0]:
# dropping the mostly empty or 'useless' columns
df = df.drop(['Unreliable', 'NonSD', 'Mass (g)', 'Density (g/mL)', 'Yelp', 'Google'], axis=1)

In [0]:
# replacing data where only a few values are missing with the feature's mean
df = df.fillna(df.mean())

In [0]:
pd.options.display.max_rows = 500
df.head(500)

,Date,Chips,Cost,Hunger,Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great,Burrito_Asada,Burrito_California,Burrito_Carnitas,Burrito_Other,Burrito_Surf & Turf
0,2016-01-18,0.0,6.490000,3.000000,20.038233,22.135765,0.786477,3.00,5.000000,3.000000,3.500000,4.000000,4.000000,4.00000,4.000000,4.000000,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0
1,2016-01-24,0.0,5.450000,3.500000,20.038233,22.135765,0.786477,2.00,3.500000,2.500000,2.500000,2.000000,4.000000,3.50000,2.500000,5.000000,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0
2,2016-01-24,0.0,4.850000,1.500000,20.038233,22.135765,0.786477,3.00,2.000000,2.500000,3.000000,4.500000,4.000000,3.00000,3.000000,5.000000,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0
3,2016-01-24,0.0,5.250000,2.000000,20.038233,22.135765,0.786477,3.00,2.000000,3.500000,3.000000,4.000000,5.000000,4.00000,4.000000,5.000000,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0
4,2016-01-27,1.0,6.590000,4.000000,20.038233,22.135765,0.786477,4.00,5.000000,4.000000,3.500000,4.500000,5.000000,2.50000,4.500000,4.000000,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,0,0,0
5,2016-01-28,0.0,6.990000,4.000000,20.038233,22.135765,0.786477,3.00,4.000000,5.000000,3.500000,2.500000,2.500000,2.50000,4.000000,1.000000,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0
6,2016-01-30,0.0,7.190000,1.500000,20.038233,22.135765,0.786477,2.00,3.000000,3.000000,2.000000,2.500000,2.500000,3.37197,2.000000,3.000000,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0
7,2016-01-30,0.0,6.990000,4.000000,20.038233,22.135765,0.786477,2.50,3.000000,3.000000,2.500000,3.000000,3.500000,3.37197,2.500000,3.000000,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0
8,2016-02-01,1.0,9.250000,3.500000,20.038233,22.135765,0.786477,2.00,4.500000,4.500000,3.500000,1.500000,3.000000,3.50000,4.000000,2.000000,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0
9,2016-02-06,0.0,6.250000,3.500000,20.038233,22.135765,0.786477,2.50,1.500000,1.500000,3.000000,4.500000,3.000000,1.50000,2.000000,4.500000,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0


In [0]:
#  Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
train = df[(df['Date'] < '1/1/2017')]
validate = df[(df['Date'] >= '1/1/2017') & (df['Date'] < '1/1/2018')]
test = df[(df['Date'] >= '1/1/2018')]

In [0]:
train['Date'].min(), train['Date'].max()

(Timestamp('2011-05-16 00:00:00'), Timestamp('2016-12-15 00:00:00'))

In [0]:
validate['Date'].min(), validate['Date'].max()

(Timestamp('2017-01-04 00:00:00'), Timestamp('2017-12-29 00:00:00'))

In [0]:
test['Date'].min(), test['Date'].max()

(Timestamp('2018-01-02 00:00:00'), Timestamp('2026-04-25 00:00:00'))

In [0]:
test = test[test['Date'] < '12/6/2019'] # drop any burrito eaten in the future
test['Date'].max()

Timestamp('2019-08-27 00:00:00')

In [0]:
# Begin with baselines for classification.
baseline = train['Great'].value_counts(normalize=True).max()
print('Zero-feature Test Accuracy: {:.1f}%'.format(baseline*100))

Zero-feature Test Accuracy: 59.1%


In [0]:
baseline = validate['Great'].value_counts(normalize=True).max()
print('Zero-feature Validation Accuracy: {:.1f}%'.format(baseline*100))

Zero-feature Validation Accuracy: 55.3%


In [0]:
# Use scikit-learn for logistic regression.
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(solver='saga', max_iter=10000)

X_train = train.drop(['Great', 'Date'], axis=1)
y_train = train['Great']
X_validate = validate.drop(['Great', 'Date'], axis=1)
y_validate = validate['Great']
X_test = test.drop(['Great', 'Date'], axis=1)
y_test = test['Great']
log_reg.fit(X_train, y_train);

In [0]:
# Get your model's validation accuracy.
from sklearn.metrics import accuracy_score
print('Validation Accuracy {:.1f}%'.format(log_reg.score(X_validate, y_validate)*100))

Validation Accuracy 83.5%


In [0]:
# Get your model's test accuracy.

In [0]:
print('Test Accuracy {:.1f}%'.format(log_reg.score(X_test, y_test)*100))

Test Accuracy 75.7%
